## Chapter 2. 파이썬스러운(pythonic) 코드

In [2]:
my_numbers = (4, 5, 3, 9)

print("음수 인덱스:", my_numbers[-1], my_numbers[-3])
print("slice 1:", my_numbers[2:5])
print("slice 2:", my_numbers[:3])
print("slice 3:", my_numbers[3:])
print("slice 4:", my_numbers[::-1])

print("\n")
print("slice index:", my_numbers[1:7:2])
print("slice object:", my_numbers[slice(1, 7, 2)])

print("\n")
interval = slice(None, 3)
print("slice object vs index:", my_numbers[interval] == my_numbers[:3])

음수 인덱스: 9 5
slice 1: (3, 9)
slice 2: (4, 5, 3)
slice 3: (9,)
slice 4: (9, 3, 5, 4)


slice index: (5, 9)
slice object: (5, 9)


slice object vs index: True


In [1]:
""" list wrapping 사례 """

class Item:
    def __init__(self, *values):
        self._value = list(values)
    
    def __len__(self):
        return len(self._value)

    def __getitem__(self, index):
        return self._value.__getitem__(index)

In [2]:
item = Item(3, 4, 5)
print(len(item))
print(item[1:])
print(type(item[1:]))

3
[4, 5]
<class 'list'>


In [7]:
Item(3,4,5)

In [70]:
""" 컨텍스트 관리자를 활용한 DB Dump 사례 """

def stop_database():
    print("systemctl stop postgresql.service")

def start_database():
    print("systemctl start postgresql.service")


class DBHandler:
    def __enter__(self):
        stop_database()
        return self

    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()


def db_backup():
    print("pg_dump database")

def main():
    with DBHandler():
        db_backup()

main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [71]:
import contextlib

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

def main():
    with db_handler():
        db_backup()

main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [72]:
import contextlib

class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()

    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@dbhandler_decorator()
def offline_backup():
    print("pg_dump database")

In [73]:
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60 # 하나의 밑줄

conn = Connector("postgresql://localhost")

conn._timeout = 70
print(conn.__dict__)

{'source': 'postgresql://localhost', '_timeout': 70}


In [74]:
class Connector:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60 # 두 개의 밑줄

conn = Connector("postgresql://localhost")

conn.__timeout # "접근할 수 없다"가 아닌 "존재하지 않는다"는 오류 발생

AttributeError: 'Connector' object has no attribute '__timeout'

In [75]:
conn._Connector__timeout = 80 # 이름이 바뀌어 있음
print(conn.__dict__)

{'source': 'postgresql://localhost', '_Connector__timeout': 80}


In [88]:
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None

class User:
    def __init__(self, username):
        self.username = username
        self._email = None

    @property
    def email(self):
        return self._email

    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없음")
        self._email = new_email

user = User("hhh.aaa")
user.email = "hhh.aaa@"

ValueError: 유효한 이메일이 아니므로 hhh.aaa@ 값을 사용할 수 없음